# CREATE CLEANED CSV CONTAINING FEATURES (INDEX, VAERS_ID, SEX, AGE_YRS, VAX_NAME)

In [3]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
import tensorflow as tf
import re
from sqlalchemy import create_engine  
from sqlalchemy import Column, String  
from sqlalchemy.ext.declarative import declarative_base  
from sqlalchemy.orm import sessionmaker
import pandas.io.sql as sqlio
import psycopg2

In [4]:
# Use psycopg2 to import cleaned table from pgadmin
conn = psycopg2.connect("dbname=VAERS user=postgres password=sql123")
sql = "select * from vaers_3;"
vaers_df = sqlio.read_sql_query(sql, conn)
conn = None

In [5]:
# Print DF
print(vaers_df.shape)
vaers_df.head(5)

(40247, 49)


,vaers_id,recvdate,state_,age_yrs,cage_yr,sex,symptom_text,died,datedied,l_threat,...,symptomversion4,symptom5,symptomversion5,vax_type,vax_manu,vax_lot,vax_dose_series,vax_route,vax_site,vax_name
0,1000000,02/04/2021,CA,NaN,NaN,M,tested positive; tested positive; This is a sp...,None,None,None,...,NaN,None,NaN,COVID19,PFIZER\BIONTECH,None,1,None,None,COVID19 (COVID19 (PFIZER-BIONTECH))
1,1000001,02/04/2021,WI,NaN,NaN,F,covid symptoms the 28th and tested positive; c...,None,None,None,...,NaN,None,NaN,COVID19,PFIZER\BIONTECH,None,1,None,None,COVID19 (COVID19 (PFIZER-BIONTECH))
2,1000003,02/04/2021,PA,29.0,NaN,F,rash and hives all over body; rash and hives a...,None,None,None,...,NaN,None,NaN,COVID19,PFIZER\BIONTECH,EJ1685,1,OT,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,1000006,02/04/2021,None,NaN,NaN,F,tested positive for covid; tested positive for...,None,None,None,...,NaN,None,NaN,COVID19,PFIZER\BIONTECH,None,1,None,None,COVID19 (COVID19 (PFIZER-BIONTECH))
4,1000007,02/04/2021,WI,53.0,NaN,M,blood sugar has been out of control ranging fr...,None,None,None,...,NaN,None,NaN,COVID19,PFIZER\BIONTECH,None,1,None,RA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [6]:
# Drop duplicates in the dataframe
vaers_df.drop_duplicates(inplace=True)
print(vaers_df.shape)
vaers_df.head(5)

(40209, 49)


,vaers_id,recvdate,state_,age_yrs,cage_yr,sex,symptom_text,died,datedied,l_threat,...,symptomversion4,symptom5,symptomversion5,vax_type,vax_manu,vax_lot,vax_dose_series,vax_route,vax_site,vax_name
0,1000000,02/04/2021,CA,NaN,NaN,M,tested positive; tested positive; This is a sp...,None,None,None,...,NaN,None,NaN,COVID19,PFIZER\BIONTECH,None,1,None,None,COVID19 (COVID19 (PFIZER-BIONTECH))
1,1000001,02/04/2021,WI,NaN,NaN,F,covid symptoms the 28th and tested positive; c...,None,None,None,...,NaN,None,NaN,COVID19,PFIZER\BIONTECH,None,1,None,None,COVID19 (COVID19 (PFIZER-BIONTECH))
2,1000003,02/04/2021,PA,29.0,NaN,F,rash and hives all over body; rash and hives a...,None,None,None,...,NaN,None,NaN,COVID19,PFIZER\BIONTECH,EJ1685,1,OT,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,1000006,02/04/2021,None,NaN,NaN,F,tested positive for covid; tested positive for...,None,None,None,...,NaN,None,NaN,COVID19,PFIZER\BIONTECH,None,1,None,None,COVID19 (COVID19 (PFIZER-BIONTECH))
4,1000007,02/04/2021,WI,53.0,NaN,M,blood sugar has been out of control ranging fr...,None,None,None,...,NaN,None,NaN,COVID19,PFIZER\BIONTECH,None,1,None,RA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [7]:
#Keep only the age, gender, died, and vax name columns for unsupervised model
vaers_df_UML = vaers_df[['vaers_id','age_yrs', 'sex', 'died', 'vax_name' ]]
vaers_df_UML.head(10)


,vaers_id,age_yrs,sex,died,vax_name
0,1000000,NaN,M,None,COVID19 (COVID19 (PFIZER-BIONTECH))
1,1000001,NaN,F,None,COVID19 (COVID19 (PFIZER-BIONTECH))
2,1000003,29.0,F,None,COVID19 (COVID19 (PFIZER-BIONTECH))
3,1000006,NaN,F,None,COVID19 (COVID19 (PFIZER-BIONTECH))
4,1000007,53.0,M,None,COVID19 (COVID19 (PFIZER-BIONTECH))
5,1000012,NaN,F,None,COVID19 (COVID19 (PFIZER-BIONTECH))
6,1000013,NaN,M,None,COVID19 (COVID19 (PFIZER-BIONTECH))
7,1000015,NaN,F,None,COVID19 (COVID19 (PFIZER-BIONTECH))
8,1000021,62.0,F,None,COVID19 (COVID19 (PFIZER-BIONTECH))
9,1000031,NaN,F,None,COVID19 (COVID19 (PFIZER-BIONTECH))


In [8]:
# Check vaers_df column types
vaers_df_UML.dtypes

vaers_id     object
age_yrs     float64
sex          object
died         object
vax_name     object
dtype: object

In [9]:
# Convert missing data in 'died' column to indicate that death has not occured ("Y"/"N")
vaers_df_UML['died'].fillna(value='N', inplace=True)
vaers_df_UML.died

C:\Users\Kanav\anaconda3\envs\PythonData2\lib\site-packages\pandas\core\series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


0        N
1        N
2        N
3        N
4        N
        ..
40242    N
40243    N
40244    N
40245    N
40246    N
Name: died, Length: 40209, dtype: object

In [10]:
# Create new DF that does not contain null values
vaers_UML = vaers_df_UML.dropna()
vaers_UML.head(10)

,vaers_id,age_yrs,sex,died,vax_name
2,1000003,29.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
4,1000007,53.0,M,N,COVID19 (COVID19 (PFIZER-BIONTECH))
8,1000021,62.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
10,1000036,38.0,M,N,COVID19 (COVID19 (PFIZER-BIONTECH))
13,1000046,86.0,M,N,COVID19 (COVID19 (PFIZER-BIONTECH))
14,1000051,70.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
15,1000051,70.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
16,1000051,70.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
19,1000074,80.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
20,1000074,80.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))


In [11]:
# Analyze 'sex' column
vaers_UML.sex.value_counts()

F    26778
M     9902
U      176
Name: sex, dtype: int64

In [12]:
# Analyze 'died' column
vaers_UML.died.value_counts()

N    34430
Y     2426
Name: died, dtype: int64

In [13]:
# Analyze 'vax_name' column
pd.options.display.max_rows = None
vaers_UML.vax_name.value_counts()

COVID19 (COVID19 (MODERNA))                              18546
COVID19 (COVID19 (PFIZER-BIONTECH))                      17460
COVID19 (COVID19 (JANSSEN))                                401
VACCINE NOT SPECIFIED (NO BRAND NAME)                       95
ZOSTER (SHINGRIX)                                           62
COVID19 (COVID19 (UNKNOWN))                                 30
INFLUENZA (SEASONAL) (FLUZONE QUADRIVALENT)                 25
PNEUMO (PNEUMOVAX)                                          23
INFLUENZA (SEASONAL) (NO BRAND NAME)                        21
VACCINE NOT SPECIFIED (OTHER)                               15
INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE QUADRIVALENT)       15
MEASLES + MUMPS + RUBELLA (MMR II)                          13
ZOSTER LIVE (ZOSTAVAX)                                      12
INFLUENZA (SEASONAL) (FLUBLOK QUADRIVALENT)                 11
VARICELLA (VARIVAX)                                          8
INFLUENZA (SEASONAL) (FLUAD QUADRIVALENT)              

In [14]:
# Filter the vax_name column for values that contain "COVID19"
vaers_UML = vaers_UML[vaers_UML['vax_name'].str.contains("COVID19")]
vaers_UML.head(10)

,vaers_id,age_yrs,sex,died,vax_name
2,1000003,29.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
4,1000007,53.0,M,N,COVID19 (COVID19 (PFIZER-BIONTECH))
8,1000021,62.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
10,1000036,38.0,M,N,COVID19 (COVID19 (PFIZER-BIONTECH))
13,1000046,86.0,M,N,COVID19 (COVID19 (PFIZER-BIONTECH))
14,1000051,70.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
15,1000051,70.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
16,1000051,70.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
19,1000074,80.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
20,1000074,80.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))


In [15]:
#Check if only desired values remain
pd.options.display.max_rows = None
vaers_UML.vax_name.value_counts()

COVID19 (COVID19 (MODERNA))            18546
COVID19 (COVID19 (PFIZER-BIONTECH))    17460
COVID19 (COVID19 (JANSSEN))              401
COVID19 (COVID19 (UNKNOWN))               30
Name: vax_name, dtype: int64

In [16]:
# Analyze values
vaers_UML.sex.value_counts()

F    26510
M     9760
U      167
Name: sex, dtype: int64

In [17]:
# Analyze values
vaers_UML.died.value_counts()

N    34084
Y     2353
Name: died, dtype: int64

In [18]:
#Tidy vax_name column by removing "COVID19" entered twice each row, and unnecessary parenthesis
vaccine_name = vaers_UML['vax_name'].astype(str)
vaccine_name.head()

2     COVID19 (COVID19 (PFIZER-BIONTECH))
4     COVID19 (COVID19 (PFIZER-BIONTECH))
8     COVID19 (COVID19 (PFIZER-BIONTECH))
10    COVID19 (COVID19 (PFIZER-BIONTECH))
13    COVID19 (COVID19 (PFIZER-BIONTECH))
Name: vax_name, dtype: object

In [19]:
#Split text
vaccine_name = vaccine_name.str.split("(", n=2, expand = True)
vaccine_name.head()

,0,1,2
2,COVID19,COVID19,PFIZER-BIONTECH))
4,COVID19,COVID19,PFIZER-BIONTECH))
8,COVID19,COVID19,PFIZER-BIONTECH))
10,COVID19,COVID19,PFIZER-BIONTECH))
13,COVID19,COVID19,PFIZER-BIONTECH))


In [20]:
# Set equal to vaccine name
vaccine_name = vaccine_name[2]

In [21]:
# Remove parenthesis at the end of each row
vaccine_name = vaccine_name.str.replace(r"\).*\)","")


In [22]:
# Check results
vaccine_name.head()

2     PFIZER-BIONTECH
4     PFIZER-BIONTECH
8     PFIZER-BIONTECH
10    PFIZER-BIONTECH
13    PFIZER-BIONTECH
Name: 2, dtype: object

In [23]:
# Check results
vaers_UML.head(15)

,vaers_id,age_yrs,sex,died,vax_name
2,1000003,29.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
4,1000007,53.0,M,N,COVID19 (COVID19 (PFIZER-BIONTECH))
8,1000021,62.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
10,1000036,38.0,M,N,COVID19 (COVID19 (PFIZER-BIONTECH))
13,1000046,86.0,M,N,COVID19 (COVID19 (PFIZER-BIONTECH))
14,1000051,70.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
15,1000051,70.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
16,1000051,70.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
19,1000074,80.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))
20,1000074,80.0,F,N,COVID19 (COVID19 (PFIZER-BIONTECH))


In [24]:
# set vax_name columns to new variable
vaers_UML['vax_name'] = vaccine_name

In [25]:
# Check results
vaers_UML.head(50)

,vaers_id,age_yrs,sex,died,vax_name
2,1000003,29.0,F,N,PFIZER-BIONTECH
4,1000007,53.0,M,N,PFIZER-BIONTECH
8,1000021,62.0,F,N,PFIZER-BIONTECH
10,1000036,38.0,M,N,PFIZER-BIONTECH
13,1000046,86.0,M,N,PFIZER-BIONTECH
14,1000051,70.0,F,N,PFIZER-BIONTECH
15,1000051,70.0,F,N,PFIZER-BIONTECH
16,1000051,70.0,F,N,PFIZER-BIONTECH
19,1000074,80.0,F,N,PFIZER-BIONTECH
20,1000074,80.0,F,N,PFIZER-BIONTECH


In [26]:
vaers_UML.drop_duplicates(inplace=True)

In [27]:
vaers_UML.head(50)

,vaers_id,age_yrs,sex,died,vax_name
2,1000003,29.0,F,N,PFIZER-BIONTECH
4,1000007,53.0,M,N,PFIZER-BIONTECH
8,1000021,62.0,F,N,PFIZER-BIONTECH
10,1000036,38.0,M,N,PFIZER-BIONTECH
13,1000046,86.0,M,N,PFIZER-BIONTECH
14,1000051,70.0,F,N,PFIZER-BIONTECH
19,1000074,80.0,F,N,PFIZER-BIONTECH
22,1000082,70.0,F,N,PFIZER-BIONTECH
24,1000103,54.0,M,N,PFIZER-BIONTECH
29,1000122,36.0,F,N,PFIZER-BIONTECH


In [28]:
#Confirm there are no null values. 
for column in vaers_UML:
    print (f"Column {column} has {vaers_UML[column].isnull().sum()} null values")

Column vaers_id has 0 null values
Column age_yrs has 0 null values
Column sex has 0 null values
Column died has 0 null values
Column vax_name has 0 null values


In [29]:
#Confirm there are no duplicate entries.
print(f"Duplicate entries: {vaers_UML['vaers_id'].duplicated().sum()}")

Duplicate entries: 9


In [30]:
#Export CSV for dashboard use, proceed to convert text strings to numerical values
vaers_UML.to_csv('VAERS_UML_cleaned.csv')

# CREATE CLEANED DATAFRAME FOR UNSUPERVISED MACHINE-LEARNING (ONLY NUMERICAL DATA)

In [31]:
#Convert sex column from object to boolean numerical values for unsupervised model
def change_sex(sex):
    if sex == "M":
        return 1
    if sex == "F":
        return 2
    else: 
        return 3
    
vaers_UML ["sex"] = vaers_UML["sex"].apply(change_sex)
vaers_UML.head(10)

,vaers_id,age_yrs,sex,died,vax_name
2,1000003,29.0,2,N,PFIZER-BIONTECH
4,1000007,53.0,1,N,PFIZER-BIONTECH
8,1000021,62.0,2,N,PFIZER-BIONTECH
10,1000036,38.0,1,N,PFIZER-BIONTECH
13,1000046,86.0,1,N,PFIZER-BIONTECH
14,1000051,70.0,2,N,PFIZER-BIONTECH
19,1000074,80.0,2,N,PFIZER-BIONTECH
22,1000082,70.0,2,N,PFIZER-BIONTECH
24,1000103,54.0,1,N,PFIZER-BIONTECH
29,1000122,36.0,2,N,PFIZER-BIONTECH


In [32]:
#Convert died column from object to boolean numerical values for unsupervised model
def change_died(died):
    if died == "N":
        return 1
    else: 
        return 2
    
vaers_UML ["died"] = vaers_UML["died"].apply(change_died)
vaers_UML.head(15)

,vaers_id,age_yrs,sex,died,vax_name
2,1000003,29.0,2,1,PFIZER-BIONTECH
4,1000007,53.0,1,1,PFIZER-BIONTECH
8,1000021,62.0,2,1,PFIZER-BIONTECH
10,1000036,38.0,1,1,PFIZER-BIONTECH
13,1000046,86.0,1,1,PFIZER-BIONTECH
14,1000051,70.0,2,1,PFIZER-BIONTECH
19,1000074,80.0,2,1,PFIZER-BIONTECH
22,1000082,70.0,2,1,PFIZER-BIONTECH
24,1000103,54.0,1,1,PFIZER-BIONTECH
29,1000122,36.0,2,1,PFIZER-BIONTECH


In [33]:
vaers_UML["vax_name"].head()

2     PFIZER-BIONTECH
4     PFIZER-BIONTECH
8     PFIZER-BIONTECH
10    PFIZER-BIONTECH
13    PFIZER-BIONTECH
Name: vax_name, dtype: object

In [34]:
#Convert vax_name column from object to numerical values for unsupervised model, representing each vaccine (Moderna, Pfizer, Janssen)
def change_vax(vax_name):
    if vax_name == "MODERNA":
        return 1
    elif vax_name == "PFIZER-BIONTECH":
        return 2
    elif vax_name == "JANSSEN":
        return 3
    else:
        return 4
vaers_UML["vax_name"] = vaers_UML["vax_name"].apply(change_vax)
vaers_UML.head(50)


,vaers_id,age_yrs,sex,died,vax_name
2,1000003,29.0,2,1,2
4,1000007,53.0,1,1,2
8,1000021,62.0,2,1,2
10,1000036,38.0,1,1,2
13,1000046,86.0,1,1,2
14,1000051,70.0,2,1,2
19,1000074,80.0,2,1,2
22,1000082,70.0,2,1,2
24,1000103,54.0,1,1,2
29,1000122,36.0,2,1,2


In [35]:
vaers_UML.head(50)

,vaers_id,age_yrs,sex,died,vax_name
2,1000003,29.0,2,1,2
4,1000007,53.0,1,1,2
8,1000021,62.0,2,1,2
10,1000036,38.0,1,1,2
13,1000046,86.0,1,1,2
14,1000051,70.0,2,1,2
19,1000074,80.0,2,1,2
22,1000082,70.0,2,1,2
24,1000103,54.0,1,1,2
29,1000122,36.0,2,1,2


CONDUCT UNSUPERVISED MODEL

In [36]:
import pandas as pd
import plotly.express as px
import hvplot.pandas
from sklearn.cluster import KMeans

In [37]:
# Initializing model 
model = KMeans(n_clusters=4, random_state=0)
model

KMeans(n_clusters=4, random_state=0)

In [38]:
# Fitting model
model.fit(vaers_UML)

KMeans(n_clusters=4, random_state=0)

In [39]:
# Get the predictions
predictions = model.predict(vaers_UML)
print(predictions)

[2 2 2 ... 2 2 2]


In [40]:
# Add a new class column to the df_iris
vaers_UML["class"] = model.labels_
vaers_UML.head(50)

,vaers_id,age_yrs,sex,died,vax_name,class
2,1000003,29.0,2,1,2,2
4,1000007,53.0,1,1,2,2
8,1000021,62.0,2,1,2,2
10,1000036,38.0,1,1,2,2
13,1000046,86.0,1,1,2,2
14,1000051,70.0,2,1,2,2
19,1000074,80.0,2,1,2,2
22,1000082,70.0,2,1,2,2
24,1000103,54.0,1,1,2,2
29,1000122,36.0,2,1,2,2


In [41]:
vaers_UML.class.value_counts()


SyntaxError: invalid syntax (<ipython-input-41-15c65e569bb7>, line 1)

In [42]:
import plotly.express as px
import hvplot.pandas

In [43]:
# Create a scatterplot of df_iris
vaers_UML.hvplot.scatter(x="vaers_id", y="age_yrs", by="class")

:NdOverlay   [class]
   :Scatter   [vaers_id]   (age_yrs)

In [45]:
# Plotting the clusters
fig = px.scatter_3d(vaers_UML, x="vaers_id", y="sex", z="age_yrs", color="died", symbol="class", size="vax_name",width=800)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [ ]:
#Use REGEX to filter for only COVID19 KEEP FOR FUTURE USE
#vax_names = vaers_UML['vax_name']
#vax_names.str.contains(regex, flags=re.IGNORECASE).sum()

In [ ]:
#Remove all values in vax_name that do not include 'COVID19' using REGEX, in order to select only the top three unique values in 
#vax_name. Therefore the only three vaccines will be three covid19 vaccines; moderna, pfizer-biontech, and janssen. 
#regex = r' \b(\w*COVID19\w*)\b'